In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install tfrecord

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 3.1 MB/s eta 0:00:00


In [1]:
# Change this line if you're using Colab to something like '/content/drive/MyDrive/TeamX/'
# where TeamX is just the clone of repository on your Google Drive
# and you have mounted the drive at /content/drive
# See the Tutorial Slides for more detail.

# Works on your local machine but not on Colab!
PROJECT_ROOT = '/content/drive/MyDrive/human-pose-prediction-in-the-wild'

# Fix this path and use this one on Colab
# PROJECT_ROOT = '/content/drive/MyDrive/TeamX'

import os
import sys
import torch
import torch.nn as nn

from os.path import join as ospj
sys.path.append(ospj(PROJECT_ROOT, 'src'))
%load_ext autoreload
%autoreload 2

In [11]:
import gc

trainer.model.cpu()
del trainer.model
del trainer

gc.collect()
torch.cuda.empty_cache()
del train_loader
del val_loader
gc.collect()
torch.cuda.empty_cache()

  0% 0/4704 [00:45<?, ?it/s]
  0% 0/4656 [00:15<?, ?it/s]


In [ ]:
""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer
from utils.config_parser import ConfigParser
import data.threeDPW as module_data
from utils.io import seed_everything

# For fair comparisons
seed_everything(100)

%aimport -ConfigParser # Due to an issue of pickle and auto_reload
config = ConfigParser.wo_args(config=ospj(PROJECT_ROOT,'cfgs/project/asim-config.json'))

datamodule = config.init_obj('train_loader', module_data)
train_loader = datamodule.get_loader()

datamodule = config.init_obj('validation_loader', module_data)
val_loader = datamodule.get_loader()

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
stats = trainer.train()
# plt.plot(stats['loss']['train'], label='train')
# plt.plot(stats['loss']['val'], label='val')
# plt.title('Classification loss history')
# plt.xlabel('Epoch')
# plt.ylabel('Classification loss')
# plt.legend()
# plt.show()


DEBUG:trainer:==> Start Training Epoch 1/20, lr=0.001000 
Train Epoch: 1 Loss: 5.457490: : 100% 4656/4656 [08:16<00:00,  9.37it/s]
DEBUG:trainer:==> Finished Epoch 1/20.
DEBUG:trainer:++> Evaluate at epoch 1 ...
  0% 0/3200 [00:00<?, ?it/s]/usr/local/lib/python3.10/dist-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at ../aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
Eval Loss: 1.143886: : 100% 3200/3200 [03:13<00:00, 16.55it/s]
DEBUG:trainer:++> Evaluate epoch 1 Finished.
INFO:trainer:Saving checkpoint: /content/drive/MyDrive/human-pose-prediction-in-the-wild/saved/models/HPPW/0716_180233/best_model.pth ...
INFO:trainer:Checkpoint saved.
INFO:trainer:Saving model with best metric at /content/drive/MyDrive/human-pose-prediction-in-the-wild/saved/models/HPPW/0716_180233/bes

In [ ]:
from prettytable import PrettyTable
def count_parameters(model):
    table = PrettyTable(["Modules", "Parameters"])
    total_params = 0
    for name, parameter in model.named_parameters():
        if not parameter.requires_grad: continue
        params = parameter.numel()
        table.add_row([name, params])
        total_params+=params
    print(table)
    print(f"Total Trainable Params: {total_params}")
    return total_params

In [7]:
from torchsummary import summary
from tqdm import tqdm
for i, (history, future) in enumerate(tqdm(val_loader)):
    output = trainer.model(history, future)





  0%|          | 0/203 [00:00<?, ?it/s]


OutOfMemoryError: ignored

In [18]:
print(output.shape)

torch.Size([12, 10, 19, 3])


In [ ]:
import torchinfo
torchinfo.summary(trainer.model)

Layer (type:depth-idx)                                                      Param #
HumanPosePredictorModel                                                     --
├─PoseEncoder: 1-1                                                          --
│    └─Sequential: 2-1                                                      --
│    │    └─PoseEncoderBlock: 3-1                                           3,153,152
│    │    └─PoseEncoderBlock: 3-2                                           3,153,152
├─VisionTransformer: 1-2                                                    768
│    └─Conv2d: 2-2                                                          590,592
│    └─Encoder: 2-3                                                         151,296
│    │    └─MultiInputSequential: 3-3                                       85,054,464
│    │    └─LayerNorm: 3-4                                                  1,536
│    └─Sequential: 2-4                                                      --
│    │    └

In [ ]:
""" Perform test
"""

checkpoint_dir = '0604_104809'
path = ospj(PROJECT_ROOT, f'saved/models/hppw/{checkpoint_dir}/best_val_model.pth')

trainer.load_model(path=path)

result = trainer.evaluate(loader=test_loader)

print(result)


In [ ]:
from models.temporal.encoder import TemporalEncoder, LocalTemporalEncoderBlock

model = TemporalEncoder(
    num_layers=3,
    num_heads=8,
    hidden_dim=256,
    mlp_dim=512
)

In [ ]:
local_feat = torch.randn(size=(32, 15, 197, 256))
global_feat = torch.randn(size=(32, 15, 256))

local_out, global_out = model(local_feat, global_feat)

In [ ]:
local_out

tensor([[[ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336],
         ...,
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6335],
         [ 0.7680,  0.1498,  2.5715,  ..., -1.7233, -7.7865,  8.6336]],

        [[-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         ...,
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401],
         [-1.2756, -7.0257, -1.1117,  ...,  4.7989, -0.6455,  3.4401]],

        [[-1.1115, -3.7773, -9.5381,  ..., -0.7923,  2.3916, -3.7397],
         [-1.1115, -3.7773, -9.5381,  ..., -0

In [ ]:
from torchsummary import summary

summary(model, input_data=[local_feat, global_feat])

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

Layer (type:depth-idx)                             Output Shape              Param #
├─MultiInputSequential: 1-1                        [-1, 197, 256]            --
|    └─TemporalEncoderBlock: 2-1                   [-1, 14, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-1         [-1, 14, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-2        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-2                   [-1, 13, 197, 256]        --
|    |    └─LocalTemporalEncoderBlock: 3-3         [-1, 13, 197, 256]        528,128
|    |    └─GlobalTemporalEncoderBlock: 3-4        [-1, 15, 256]             527,104
|    └─TemporalEncoderBlock: 2-3                   [-1, 197, 256]            --
|    |    └─LocalTemporalEncoderBlock: 3-5         [-1, 197, 256]            528,128
|    |    └─GlobalTemporalEncoderBlock: 3-6        [-1, 15, 256]             527,104
Total params: 3,165,696
Trainable params: 3,165,696
Non-trainable params: 0
Total mul

In [ ]:

""" Initialize the trainer and model
"""
import matplotlib.pyplot as plt

from trainers.hppw_trainer import HPPWTrainer

trainer = HPPWTrainer(config=config, train_loader=train_loader, eval_loader=val_loader)
# stats = trainer.train()

plt.plot(stats['loss']['train'], label='train')
plt.plot(stats['loss']['val'], label='val')
plt.title('Classification loss history')
plt.xlabel('Epoch')
plt.ylabel('Classification loss')
plt.legend()
plt.show()


KeyError: 'wandb'